In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [3]:
df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [4]:
df = pd.concat([df,pd.get_dummies(df.kitchenqual, prefix="kitchenqual", drop_first=True)], axis=1)
df = pd.concat([df,pd.get_dummies(df.street, prefix="street", drop_first=True)], axis=1)
dummy_column_names = list(pd.get_dummies(df.kitchenqual, prefix="kitchenqual", drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(df.street, prefix="street", drop_first=True).columns)

In [5]:
X = df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf']+ dummy_column_names]
Y = df['saleprice']

In [6]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.784
Method:                 Least Squares   F-statistic:                     589.4
Date:                Tue, 07 Jan 2020   Prob (F-statistic):               0.00
Time:                        10:37:20   Log-Likelihood:                -17421.
No. Observations:                1460   AIC:                         3.486e+04
Df Residuals:                    1450   BIC:                         3.491e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -1.044e+04   1.75e+04     -0.596      0.551   -4.48e+04    2.39e+04
overallqual     1.814e+04   1151.273     15.760      0.000    1.59e+04    2.04e+04
grlivarea         44.4179      2.371     18.732      0.000      39.766      49.069
garagecars      1.502e+04   2887.750      5.201      0.000    9354.996    2.07e+04
garagearea        10.0532      9.985      1.007      0.314      -9.532      29.639
totalbsmtsf       26.9404      2.787      9.668      0.000      21.474      32.407
kitchenqual_Fa -6.073e+04   8007.211     -7.584      0.000   -7.64e+04    -4.5e+04
kitchenqual_Gd -4.819e+04   4310.127    -11.180      0.000   -5.66e+04   -3.97e+04
kitchenqual_TA -6.262e+04   4910.143    -12.754      0.000   -7.23e+04    -5.3e+04
street_Pave     6106.6299   1.53e+04      0.400      0.689   -2.38e+04     3.6e+04
==============================================================================
Omnibus:                      551.442   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            56050.036
Skew:                          -0.770   Prob(JB):                         0.00
Kurtosis:                      33.315   Cond. No.                     4.68e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.68e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### The model can explain 78.4% of the variation in the houseprices. P-value and F-stats are 0.00 and 589 respectively. AIC and BIC are 3.486e+04, 3.491e+04 respectively. 

#### There is 22.6% of the variation unexplained by the model in our target so it can be better. 

In [8]:
X = df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf']+ dummy_column_names]
Y = np.log1p(df['saleprice'])

In [9]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.805
Method:                 Least Squares   F-statistic:                     671.7
Date:                Tue, 07 Jan 2020   Prob (F-statistic):               0.00
Time:                        10:47:21   Log-Likelihood:                 467.81
No. Observations:                1460   AIC:                            -915.6
Df Residuals:                    1450   BIC:                            -862.8
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             10.7393      0.084    128.401      0.000      10.575      10.903
overallqual        0.1043      0.005     18.987      0.000       0.094       0.115
grlivarea          0.0002   1.13e-05     17.722      0.000       0.000       0.000
garagecars         0.0987      0.014      7.159      0.000       0.072       0.126
garagearea      3.496e-05   4.77e-05      0.733      0.463   -5.85e-05       0.000
totalbsmtsf        0.0001   1.33e-05     10.171      0.000       0.000       0.000
kitchenqual_Fa    -0.2487      0.038     -6.508      0.000      -0.324      -0.174
kitchenqual_Gd    -0.0511      0.021     -2.485      0.013      -0.091      -0.011
kitchenqual_TA    -0.1490      0.023     -6.356      0.000      -0.195      -0.103
street_Pave        0.1130      0.073      1.552      0.121      -0.030       0.256
==============================================================================
Omnibus:                      881.648   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24206.921
Skew:                          -2.314   Prob(JB):                         0.00
Kurtosis:                      22.404   Cond. No.                     4.68e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.68e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### just log transforming the target variable (since it wasn't normally distribute) increases accuracy by 0.015. 

In [12]:
X = df[['overallqual', 'grlivarea', 'garagecars','totalbsmtsf','kitchenqual_Fa', 'kitchenqual_Gd','kitchenqual_TA', '']]
Y = np.log1p(df['saleprice'])

In [13]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.806
Model:                            OLS   Adj. R-squared:                  0.805
Method:                 Least Squares   F-statistic:                     862.7
Date:                Tue, 07 Jan 2020   Prob (F-statistic):               0.00
Time:                        10:50:53   Log-Likelihood:                 466.41
No. Observations:                1460   AIC:                            -916.8
Df Residuals:                    1452   BIC:                            -874.5
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             10.8526      0.042    258.681      0.000      10.770      10.935
overallqual        0.1045      0.005     19.054      0.000       0.094       0.115
grlivarea          0.0002   1.13e-05     17.879      0.000       0.000       0.000
garagecars         0.1059      0.008     13.108      0.000       0.090       0.122
totalbsmtsf        0.0001    1.3e-05     10.497      0.000       0.000       0.000
kitchenqual_Fa    -0.2534      0.038     -6.649      0.000      -0.328      -0.179
kitchenqual_Gd    -0.0519      0.021     -2.528      0.012      -0.092      -0.012
kitchenqual_TA    -0.1506      0.023     -6.438      0.000      -0.197      -0.105
==============================================================================
Omnibus:                      875.399   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23072.740
Skew:                          -2.304   Prob(JB):                         0.00
Kurtosis:                      21.922   Cond. No.                     2.40e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.4e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
df = pd.concat([df,pd.get_dummies(df.mszoning, prefix="mszoning")], axis=1)
dummycolumns = list(pd.get_dummies(df.mszoning, prefix="mszoning").columns)

In [16]:
X = df[['overallqual', 'grlivarea', 'garagecars','totalbsmtsf','kitchenqual_Fa', 'kitchenqual_Gd','kitchenqual_TA']+ 
       dummycolumns]
Y = np.log1p(df['saleprice'])

In [17]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     665.0
Date:                Tue, 07 Jan 2020   Prob (F-statistic):               0.00
Time:                        10:53:49   Log-Likelihood:                 582.87
No. Observations:                1460   AIC:                            -1142.
Df Residuals:                    1448   BIC:                            -1078.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                9.0029      0.034    266.200      0.000       8.937       9.069
overallqual          0.0994      0.005     19.278      0.000       0.089       0.110
grlivarea            0.0002   1.05e-05     19.555      0.000       0.000       0.000
garagecars           0.0985      0.008     13.095      0.000       0.084       0.113
totalbsmtsf          0.0001   1.23e-05      9.106      0.000     8.8e-05       0.000
kitchenqual_Fa      -0.2367      0.035     -6.706      0.000      -0.306      -0.167
kitchenqual_Gd      -0.0680      0.019     -3.569      0.000      -0.105      -0.031
kitchenqual_TA      -0.1555      0.022     -7.183      0.000      -0.198      -0.113
mszoning_C (all)     1.4408      0.044     32.610      0.000       1.354       1.527
mszoning_FV          1.9543      0.023     85.760      0.000       1.910       1.999
mszoning_RH          1.8666      0.036     51.713      0.000       1.796       1.937
mszoning_RL          1.9522      0.014    134.956      0.000       1.924       1.981
mszoning_RM          1.7890      0.016    111.065      0.000       1.757       1.821
==============================================================================
Omnibus:                      857.103   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27912.440
Skew:                          -2.162   Prob(JB):                         0.00
Kurtosis:                      23.979   Cond. No.                     1.42e+19
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.74e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### Both the R and R-squared increased in the second model 
#### it also has a lower F- statistics with a similar p-value
#### The AIC and BIC are lower than the first model as well 

#### this model explains 83.5% of the variation in our target variable